We're going to use a custom Cython implementation of the median filter.

In [1]:
import median_filter

In [2]:
import itertools
import numpy as np
import pandas as pd
from scipy import ndimage as ndi
from skimage import measure


l = np.arange(480) - 1
l[0] = 0
r = np.arange(480) + 1
r[-1] = r[-2]
b = np.arange(640) - 1
b[0] = 0
u = np.arange(640) + 1
u[-1] = u[-2]


def expand(mask):
    new = mask.copy()
    
    for shift in [l, r]:
        new |= mask[shift, :]
    
    for shift in [b, u]:
        new |= mask[:, shift]

    return new


def get_region(img, r, c, w):
    """Returns the square of length width with (r, c) being at the center."""
    return img[
        max(r - w, 0) : min(r + w + 1, img.shape[0]),
        max(c - w, 0) : min(c + w + 1, img.shape[1])
    ]


def remove_anomalies(img):

    mask = np.ones_like(img, dtype=bool)

    for i, j in itertools.product([-1, 0, 1], [-1, 0, 1]):
        if i == j == 0:
            continue
        shift = ndi.shift(np.asarray(img, dtype=int), (i, j), mode='reflect')
        np.logical_and(mask, (img - shift) > 45, out=mask)
    
    for r, c in np.argwhere(mask):
        region = get_region(img, r, c, 1)
        img[r, c] = region.mean()
    
    return img


def find_interesting_pixels(img):

    med = median_filter.quantile_filter(img, 20, .5)
    mask = img > med + 6
    
    #local_maxes = ndi.maximum_filter(img, size=(12, 18))
    #mask = img == local_maxes
    
    labels = measure.label(expand(mask))
    
    return pd.DataFrame(
        [
            [*region.centroid, region.area, region.eccentricity, region.solidity]
            for region in measure.regionprops(labels)
        ],
        columns=['r', 'c', 'area', 'eccentricity', 'solidity']
    )

Example.

In [3]:
from PIL import Image

img = np.asarray(Image.open('data/spotGEO/train/10/1.png')).copy()
find_interesting_pixels(img).shape

(550, 5)

In [4]:
(574, 5)

(574, 5)

Do it for each image.

In [5]:
import pathlib
from joblib import Parallel, delayed
import tqdm

def f(part, seq, frame):
    img = np.array(Image.open(frame))
    img = remove_anomalies(img)
    return find_interesting_pixels(img).assign(part=part, sequence=int(seq.name), frame=int(frame.stem))

interesting = Parallel(n_jobs=4)(
    delayed(f)(part, seq, frame)
    for part in ['train', 'test']
    for seq in tqdm.tqdm(list(pathlib.Path(f'data/spotGEO/{part}').glob('*')), position=0)
    for frame in seq.glob('*.png')
)

interesting = pd.concat(interesting)
interesting = interesting.set_index(['part', 'sequence', 'frame']).sort_index()
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 5120/5120 [58:16<00:00,  1.46it/s]  


Average number of interesting regions per image.

In [6]:
interesting.groupby(['part', 'sequence', 'frame']).size().mean()

400.2365625

Percentage of pixels this represents.

In [7]:
f'{len(interesting) / (640 * 480 * interesting.index.get_level_values("sequence").nunique() * 5):%}'

'0.162857%'

Load the provided annotations.

In [8]:
import json
import pandas as pd

sats = []

with open('data/spotGEO/train_anno.json') as f:
    for ann in json.load(f):
        for i, coords in enumerate(ann['object_coords']):
            sats.append({
                'sequence': ann['sequence_id'],
                'frame': ann['frame'],
                'satellite': i + 1,
                'r': int(coords[1] + .5),
                'c': int(coords[0] + .5),
            })
    
sats = pd.DataFrame(sats)
sats = sats.set_index(['sequence', 'frame', 'satellite'])
sats.head()

r    c
sequence frame satellite          
1        1     1          237  502
               2          222  490
               3          129  141
         2     1          214  530
               2          199  518

Now let's annotate each interesting region.

In [9]:
from scipy import optimize
from scipy.spatial import distance

def assign_labels(interesting, satellites):
    
    # Compute the distance between each satellite and each interesting location,
    # thus forming a bipartite graph
    distances = distance.cdist(satellites, interesting, metric='cityblock')
    
    # Guess which locations correspond to which satellites
    row_ind, col_ind = optimize.linear_sum_assignment(distances)

    # Each satellite is assigned, but some of them may be too distant to be likely
    likely = distances[row_ind, col_ind] < 8
    
    labels = np.full(len(interesting), False, dtype=bool)
    labels[col_ind[likely]] = True
    return labels

Example.

In [10]:
interesting.loc['train', 1, 1].head()

r           c  area  eccentricity  solidity
part  sequence frame                                                     
train 1        1       5.333333  232.437500    96      0.980202  0.750000
               1       2.923077  399.153846    13      0.946252  0.764706
               1       3.000000  366.000000     5      0.000000  1.000000
               1      15.784615  160.369231   130      0.976976  0.710383
               1       7.500000  248.500000     8      0.816497  1.000000

In [11]:
assign_labels(interesting.loc['train', 10, 1][['r', 'c']], sats.loc[10, 1][['r', 'c']])

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,

Now assign labels for each frame.

In [12]:
labels = pd.Series(dtype=bool, index=interesting.loc['train'].index)

for (sequence, frame), locations in tqdm.tqdm(interesting.loc['train'].groupby(['sequence', 'frame']), position=0):
    try:
        satellites = sats.loc[sequence, frame]
    except KeyError:
        continue
    labels.loc[sequence, frame] = assign_labels(locations[['r', 'c']], satellites[['r', 'c']])
    
interesting['is_satellite'] = None
interesting.loc['train', 'is_satellite'] = labels.values
interesting.to_pickle('data/interesting.pkl')

100%|██████████| 6400/6400 [01:15<00:00, 84.81it/s] 


Determine the amount of satellites that got assigned.

In [13]:
interesting.loc['train']['is_satellite'].sum() / len(sats)

0.8603302097278

In [14]:
0.8138331102186523

0.8138331102186523

Next, head to [Solution.ipynb](Solution.ipynb).